<a href="https://colab.research.google.com/github/v1nybarreto/desafio-pleno-data-scientist-ai/blob/setup/notebooks/01_configuracao_e_teste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import auth
auth.authenticate_user()
print("✅ Autenticado no Google Colab.")

✅ Autenticado no Google Colab.


In [3]:
from google.cloud import bigquery

PROJECT_ID = "genai-rio"
client = bigquery.Client(project=PROJECT_ID)
print("✅ BigQuery client ok. Projeto:", client.project)

✅ BigQuery client ok. Projeto: genai-rio


In [6]:
from google.cloud.bigquery import QueryJobConfig

def run_query(sql: str, dry_run: bool = False, max_bytes_billed: int | None = None):
    """
    Executa SQL no BigQuery com opção de dry-run para estimar bytes.
    - dry_run=True: somente estima bytes processados (não executa).
    - max_bytes_billed: limite de bytes (proteção quando houver billing).
    Retorna: DataFrame (quando executado) ou None (quando dry-run).
    """
    job_config = QueryJobConfig(dry_run=dry_run, use_query_cache=False)
    if max_bytes_billed is not None:
        job_config.maximum_bytes_billed = max_bytes_billed

    job = client.query(sql, job_config=job_config)
    if dry_run:
        print(f"📏 DRY RUN — bytes estimados: {job.total_bytes_processed:,}")
        return None
    df = job.to_dataframe()
    print(f"✅ Executado. Linhas retornadas: {len(df)}")
    return df

In [7]:
sql = "SELECT 1 AS ok"
run_query(sql, dry_run=True)  # estimativa
df = run_query(sql)           # execução
df

📏 DRY RUN — bytes estimados: 0
✅ Executado. Linhas retornadas: 1


,ok
0,1


In [8]:
sql = """
SELECT COUNT(*) AS total_bairros
FROM `datario.dados_mestres.bairro`
"""
run_query(sql, dry_run=True)
run_query(sql)

📏 DRY RUN — bytes estimados: 0
✅ Executado. Linhas retornadas: 1


,total_bairros
0,164


In [9]:
TEST_DATE = "2024-11-28"  # data usada nos testes do enunciado

sql = f"""
SELECT COUNT(*) AS chamados_no_dia
FROM `datario.adm_central_atendimento_1746.chamado`
WHERE data_particao = DATE('{TEST_DATE}')
"""
run_query(sql, dry_run=True)  # deve mostrar bytes bem menores que sem filtro
run_query(sql)

📏 DRY RUN — bytes estimados: 540,824
✅ Executado. Linhas retornadas: 1


,chamados_no_dia
0,0


In [10]:
sql = f"""
SELECT b.nome AS bairro, COUNT(*) AS qtde
FROM `datario.adm_central_atendimento_1746.chamado` c
JOIN `datario.dados_mestres.bairro` b
  ON c.id_bairro = b.id_bairro
WHERE c.data_particao = DATE('{TEST_DATE}')
GROUP BY bairro
ORDER BY qtde DESC
LIMIT 10
"""
run_query(sql, dry_run=True)
run_query(sql)

📏 DRY RUN — bytes estimados: 826,695
✅ Executado. Linhas retornadas: 0


,bairro,qtde


In [11]:
sql = """
SELECT partition_id, total_rows
FROM `datario.adm_central_atendimento_1746`.INFORMATION_SCHEMA.PARTITIONS
WHERE table_name = 'chamado'
ORDER BY partition_id DESC
LIMIT 10
"""
# Pode falhar se o INFORMATION_SCHEMA tiver restrições no dataset público; é apenas uma checagem adicional.
try:
    run_query(sql, dry_run=True)
    run_query(sql)
except Exception as e:
    print("ℹ️ Informação de partições indisponível neste dataset público (ok). Detalhe:", e)

📏 DRY RUN — bytes estimados: 10,485,760
✅ Executado. Linhas retornadas: 10
